In [ ]:
import numpy as np
import scipy.sparse as sp
import scipy.sparse.linalg as sla
import time

def build_finesse_solver(omega, n_phys=501, n_pml=100):
    """
    V4 Master Solver: Frequency-Invariant CFS-PML.
    Optimized for omega 16 through 128.
    """
    n_total = n_phys + 2 * n_pml
    h = 1.0 / (n_phys - 1)
    k2 = omega**2
    p = 3  # Cubic ramp for optimal smoothness/stability balance
    
    # Frequency-Invariant Finesse Parameters
    # alpha_ratio = 0.2 means alpha = 0.2 * omega
    # eta = 2.5 is the absorption strength
    complex_factor = 2.5 / (0.2 + 1j)
    
    s_profile = np.ones(n_total, dtype=complex)
    ds_profile = np.zeros(n_total, dtype=complex)
    
    for i in range(n_total):
        d = max(0, n_pml - i, n_pml - (n_total - 1 - i))
        if d > 0:
            t = d / n_pml
            s_profile[i] = 1.0 + complex_factor * (t**p)
            dt_dx = (1 / (n_pml * h)) * (-1 if i < n_pml else 1)
            ds_profile[i] = complex_factor * p * (t**(p-1)) * dt_dx

    # 4th-Order Finite Difference Coefficients
    ax = 1.0 / (s_profile**2)
    gm = -ds_profile / (s_profile**3)
    c1, c2 = 1/(12*h), 1/(12*h**2)
    
    A = sp.lil_matrix((n_total**2, n_total**2), dtype=complex)
    for j in range(n_total):
        bj, ej = ax[j], gm[j]
        for i in range(n_total):
            row = i + j * n_total
            ai, gi = ax[i], gm[i]
            
            # Center point
            A[row, row] = ai*(-30*c2) + bj*(-30*c2) + k2
            
            # 4th-order stencils (Inner points)
            if 1 < i < n_total-2:
                A[row, row-1] = ai*(16*c2) - gi*(8*c1)
                A[row, row-2] = ai*(-c2) + gi*(c1)
                A[row, row+1] = ai*(16*c2) + gi*(8*c1)
                A[row, row+2] = ai*(-c2) - gi*(c1)
            if 1 < j < n_total-2:
                A[row, row-n_total] = bj*(16*c2) - ej*(8*c1)
                A[row, row-2*n_total] = bj*(-c2) + ej*(c1)
                A[row, row+n_total] = bj*(16*c2) + ej*(8*c1)
                A[row, row+2*n_total] = bj*(-c2) - ej*(c1)
                
    return A.tocsc(), n_total, n_pml

In [ ]:
omegas = [16.0, 32.0, 64.0, 128.0]
N_PHYS = 501

print(f"{'Omega':<8} | {'Peak':<10} | {'Reflection':<12} | {'Time (s)':<8}")
print("-" * 50)

for w in omegas:
    start_t = time.time()
    A, n_tot, npml = build_finesse_solver(w, n_phys=N_PHYS, n_pml=100)
    
    # Solve for a centered point source
    f = np.zeros(n_tot**2, dtype=complex)
    mid = (n_tot // 2) * (1 + n_tot)
    f[mid] = 1.0 / ( (1.0/(N_PHYS-1))**2 )
    
    solver = sla.splu(A)
    u_vec = solver.solve(f)
    u_full = u_vec.reshape(n_tot, n_tot)
    u_phys = u_full[npml:-npml, npml:-npml]
    
    peak = np.max(np.abs(u_phys))
    refl = np.mean(np.abs(u_phys[0, :])) / peak
    elapsed = time.time() - start_t
    
    print(f"{w:<8.1f} | {peak:<10.4f} | {refl:<12.2e} | {elapsed:<8.2f}")

In [ ]:
def generate_dataset(omega, num_samples=50, n_phys=501):
    A, n_tot, npml = build_finesse_solver(omega, n_phys=n_phys, n_pml=100)
    solver = sla.splu(A)
    h = 1.0 / (n_phys - 1)
    
    data_x = [] # Sources
    data_y = [] # Solutions (Wavefields)
    
    print(f"Generating {num_samples} samples for Omega={omega}...")
    
    for s in range(num_samples):
        f = np.zeros((n_tot, n_tot), dtype=complex)
        # Random multi-source (3 to 6 sources)
        for _ in range(np.random.randint(3, 7)):
            ry, rx = np.random.randint(npml+20, n_tot-npml-20, 2)
            f[ry, rx] = (np.random.randn() + 1j*np.random.randn()) / h**2
        
        u_vec = solver.solve(f.flatten())
        u_phys = u_vec.reshape(n_tot, n_tot)[npml:-npml, npml:-npml]
        f_phys = f[npml:-npml, npml:-npml]
        
        data_x.append(f_phys.astype(np.complex64))
        data_y.append(u_phys.astype(np.complex64))
        
        if (s+1) % 10 == 0: print(f" Done {s+1}/{num_samples}")

    filename = f"helmholtz_dataset_w{int(omega)}.npz"
    np.savez_compressed(filename, src=np.array(data_x), wave=np.array(data_y))
    print(f"Dataset saved as {filename}\n")

# Run generation
for w in [16.0, 32.0, 64.0]:
    generate_dataset(w, num_samples=20) # Start with 20 for testing